In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# GET THE DATAFRAME.

import pandas as pd

df_path = "../data/raw/debunking_review.tsv"
# df_path = "../../data/raw/hedges-all.tsv"
# df_path = "../../data/raw/studytype_multiclass.tsv"
df = pd.read_csv(df_path, sep="\t")

from eppi_text_classification import (
    get_features,
    get_labels,
    get_tfidf_and_names,
)

word_features = get_features(
    df,
    # title_key="ti",
    # abstract_key="ab",
    # title_key="TI",
    # abstract_key="AB",
)

labels = get_labels(
    df,
    # label_column_name="is_rct",
    # positive_class_value="1",
    # label_column_name="Class",
    # positive_class_value="CaseControlStudies",
)
tfidf_scores, feature_names = get_tfidf_and_names(word_features)

number of processes: 12
number of processes: 12


In [3]:
glm_params = dict(
    objective="binary:logistic",
    eval_metric="logloss",
    # colsample_bytree=1,
    n_jobs=-1,
    # scale_pos_weight=42,
    scale_pos_weight=27,
    reg_lambda=0.00007636592766577953,
    reg_alpha=0.0000834292094431529,
    learning_rate=0.54507787503025981,
    # learning_rate=0.0000054507787503025981,
    # n_estimators=557,
    n_estimators=500,
    # min_child_weight=1,
    # gamma=6.384905359051449e-10,
    # max_depth=10,
    # booster="gblinear",
    # sample_type="weighted",
    # verbosity=2,
    validate_parameters=True,
    # callbacks=[es],
    enable_categorical=False,
    booster="gblinear",
)

lgbm_params = dict(
    verbosity=-1,
    boosting_type="gbdt",
    max_depth=4,
    min_child_samples=30,
    learning_rate=0.1245928917280467,
    num_leaves=6,
    n_estimators=1000,
    subsample_for_bin=20000,
    subsample=1.0,
    objective="binary",
    scale_pos_weight=27,
    min_split_gain=2.52381000405986e-06,
    min_child_weight=5.544840854157574e-06,
    reg_alpha=3.629909607712911e-05,
    reg_lambda=3.671749546766942e-05,
    n_jobs=-1,
    # linear_tree=True,
    metric="auc",
    # max_bin=255,
    # tree_method="exact",
    # grow_policy="lossguide",
    # sampling_method="uniform",
)

rf_params = dict(
    class_weight="balanced",
    n_estimators=1000,
    bootstrap=False,
    n_jobs=-1,
)

In [6]:
from eppi_text_classification.model_stability import predict_cv_metrics_per_model
from eppi_text_classification.plots.box_plot import generate_box_plot_html

model_list = ["xgboost", "lightgbm", "RandomForestClassifier"]
model_params = [glm_params, lgbm_params, rf_params]

auc_scores_per_model = predict_cv_metrics_per_model(
    tfidf_scores=tfidf_scores,
    labels=labels,
    model_names=model_list,
    model_params_list=model_params,
    nfolds=5,
    num_cv_repeats=1,
)


In [7]:
model_list2 = [
    "xgboost",
    "lightgbm",
    "RandomForestClassifier",
    "xgboost2",
    "lightgbm2",
    "RandomForestClassifier2",
    "xgboost3",
    "lightgbm3",
    "RandomForestClassifier3",
    "xgboost4",
    "lightgbm4",
    "RandomForestClassifier4",
    "xgboost5",
    "lightgbm5",
    "RandomForestClassifier5",
    "xgboost6",
    "lightgbm6",
    "RandomForestClassifier6",
    "xgboost7",
    "lightgbm7",
    "RandomForestClassifier7",
]
generate_box_plot_html(
    [
        *auc_scores_per_model,
        *auc_scores_per_model,
        *auc_scores_per_model,
        *auc_scores_per_model,
        *auc_scores_per_model,
        *auc_scores_per_model,
        *auc_scores_per_model,
    ],
    model_list2,
    "AUC",
    "Model",
    "AUC",
    "boxplot",
    "box_plot.html",
)

HTML file saved at: box_plot.html


In [123]:
single_model = ["xgboost"]

generate_box_plot_html(
    [
        auc_scores_per_model[0],
        auc_scores_per_model[2],
    ],
    ["xgboost", "RandomForest"],
    "AUC",
    "Model",
    "AUC",
    "boxplot",
    "box_plot.html",
)

HTML file saved at: box_plot.html


In [8]:
from eppi_text_classification.model_stability import predict_cv_scores

fold_scores = predict_cv_scores(
    tfidf_scores=tfidf_scores,
    labels=labels,
    model_name="xgboost",
    model_params=glm_params,
    nfolds=15,
    num_cv_repeats=1,
)
fold_scores = [list(fold) for fold in fold_scores]

In [9]:
generate_box_plot_html(
    fold_scores,
    ["1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15"],
    "AUC",
    "Fold",
    "AUC",
    "boxplot",
    "box_plot.html",
)

HTML file saved at: box_plot.html
